# Model calibration
This notebook contains a calibration of the model for the current number of agents. Usually ABMs are not linearly scalable and therefore this procedure should be followed for model calibration, to avoid unnecessary long start-up time of the model and thus a wastage of computing power. Because the RIVM stakeholders mentioned during one of the final meetings that they were especially interested in the process of model-building, this notebook is an annex to assist for when the model is built. Note that in case this model will be extended, this notebook will require to be extended, since it only includes current variables. However, most certainly the process stays the same.

Calibration outcomes of the model need to be implemented directly in the agents manually. Due to ABMs being stochastic, initial values will not be perfect but provide a good starting point. First we will be importing the model in the notebook and set it up in such a way that all policies and scenarios are turned off to see whether we can get the model in some sort of equilibrium.

In [1]:
from model.cepai_model import *

levers = {
    "L1": 0.0,  # Minimal requirement for reused parts
    "L2": 0.0,  # Minimal requirement for high-quality plastic
    "L3": 0.0,  # Use better solvable cohesives
    "L4": 1.0,  # Include externality for virgin plastic
    "L5": 0.05   # Minimal requirement for recyclate, calibrated for 5% according to Volvo.
        }

uncertainties = {
    "X1": 0.0,  # Annual increase factor of oil price
    "X2": 0.0,  # Annual probability for global oil shock
    "X3": 0.0   # Annual increase of recycling efficiency
        }

In [2]:
model = CEPAIModel(levers=levers, uncertainties=uncertainties)

Simulation starting...


In [3]:
model_run = model.run(steps=50, time_tracking=True)


KeyboardInterrupt



In [ ]:
model_run

In [3]:
for _ in range(50):
    print(_)
    model.step()

In [6]:
for PM in model.all_agents[PartsManufacturer]:
    print(PM.demand)

{<Component.VIRGIN: 1>: 308.6215907823368, <Component.RECYCLATE_LOW: 2>: 17.378409217663204, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 326}
{<Component.VIRGIN: 1>: 133.3741259465767, <Component.RECYCLATE_LOW: 2>: 7.625874053423297, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 141}
{<Component.VIRGIN: 1>: 80.65461554329241, <Component.RECYCLATE_LOW: 2>: 4.345384456707594, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 85}


In [7]:
for CM in model.all_agents[CarManufacturer]:
    # print(CM.stock)
    print(CM.demand)
    print(CM.sold_volume)

{<Component.VIRGIN: 1>: 0.0, <Component.RECYCLATE_LOW: 2>: 0.0, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 308, <Component.CARS: 6>: 77}
{'last': 77, 'second_last': 0}
{<Component.VIRGIN: 1>: 0.0, <Component.RECYCLATE_LOW: 2>: 0.0, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 348, <Component.CARS: 6>: 87}
{'last': 87, 'second_last': 0}
{<Component.VIRGIN: 1>: 0.0, <Component.RECYCLATE_LOW: 2>: 0.0, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 228, <Component.CARS: 6>: 57}
{'last': 57, 'second_last': 0}
{<Component.VIRGIN: 1>: 0.0, <Component.RECYCLATE_LOW: 2>: 0.0, <Component.RECYCLATE_HIGH: 3>: 0.0, <Component.PARTS: 4>: 268, <Component.CARS: 6>: 67}
{'last': 67, 'second_last': 0}


In [73]:
model.step()

In [78]:
counter = 0
counter_to_buy = 0
counter_to_retrieve = 0
for agent in model.all_agents[User]:
    if agent.stock[Component.CARS]:
        counter += 1
        car = agent.stock[Component.CARS][0]
        print(car.state)
    else:
        print(agent.demand[Component.CARS])
        if agent.demand[Component.CARS] == 1:
            counter_to_buy += 1

        else:
            counter_to_retrieve += 1

CarState.END_OF_LIFE
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
CarState.FUNCTIONING
1
CarState.FUNCTIONING
CarState.BROKEN
CarState.BROKEN
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
1
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
CarState.BROKEN
CarState.FUNCTIONING
CarState.END_OF_LIFE
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
1
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.END_OF_LIFE
CarState.FUNCTIONING
CarState.BROKEN
CarState.END_OF_LIFE
CarState.BROKEN
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarState.FUNCTIONING
CarSt

In [80]:
print(counter)
print(counter_to_buy)
print(counter_to_retrieve)
print(counter+counter_to_retrieve+counter_to_buy)

932
44
24
1000


In [60]:
len(model.all_agents[User])

1000

In [89]:
for garage in model.all_agents[Garage]:
    #print(garage.demand[Component.PARTS])
    print(len(garage.stock[Component.PARTS]))

52
0


In [90]:
cars_tb_repaired = 0
for garage in model.all_agents[Garage]:
    cars_tb_repaired += len(garage.stock[Component.CARS])
    print("Parts in stock")
    print(len(garage.stock[Component.PARTS]))
    print("Cars tb repaired")
    print(len(garage.stock[Component.CARS]))
    print("Cars tb recycled")
    print(len(garage.stock[Component.CARS_FOR_RECYCLER]))
    print("Cars tb dismantled")
    print(len(garage.stock[Component.CARS_FOR_DISMANTLER]))

Parts in stock
52
Cars tb repaired
0
Cars tb recycled
0
Cars tb dismantled
0
Parts in stock
0
Cars tb repaired
18
Cars tb recycled
44
Cars tb dismantled
0


In [43]:
print(cars_tb_repaired)

699
